In [ ]:
! pip install pandas
! pip install requests
! pip install beautifulsoup4
! pip install matplotlib
! pip install mlxtend

In [ ]:
import requests
import json
import json

try:
    with open("jobs.json", 'r', encoding='utf-8') as file:
        jobs = json.load(file)

except FileNotFoundError:
    response = requests.get("https://www.getonbrd.com/api/v0/search/jobs", params={
        "query": "Software engineering",
        "per_page": 100,
        "page": 1,
    })

    jobs = response.json()["data"]

    with open("jobs.json", 'w', encoding='utf-8') as file:
        json.dump(jobs, file, ensure_ascii=False, indent=4)


def guardar_en_json(array_de_objetos, nombre_archivo):
    
    with open(nombre_archivo, 'w', encoding='utf-8') as archivo_json:
        json.dump(array_de_objetos, archivo_json, ensure_ascii=False, indent=4)


guardar_en_json(jobs, 'jobs.json')

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

def transform_jobs_to_dataframe(jobs):
    
    structured_data = []
    for job in jobs:
        job_data = {
            "id": job["id"],
            "title": job["attributes"]["title"],
            "description": BeautifulSoup(job["attributes"].get("description", ""), 'html.parser').get_text(),
            "projects": BeautifulSoup(job["attributes"].get("projects", ""), 'html.parser').get_text(),
            "functions": BeautifulSoup(job["attributes"].get("functions", ""), 'html.parser').get_text(),
            "benefits": BeautifulSoup(job["attributes"].get("benefits", ""), 'html.parser').get_text(),
            "desirable": BeautifulSoup(job["attributes"].get("desirable", ""), 'html.parser').get_text(),
            "seniority": job["attributes"].get("seniority", ""),
            "remote": job["attributes"]["remote"],
            "remote_modality": job["attributes"].get("remote_modality", ""),
            "remote_zone": job["attributes"].get("remote_zone", ""),
            "countries": job["attributes"]["countries"],
            "lang": job["attributes"]["lang"],
            "category_name": job["attributes"]["category_name"],
            "perks": job["attributes"]["perks"],
            "min_salary": job["attributes"].get("min_salary"),
            "max_salary": job["attributes"].get("max_salary"),
            "published_at": job["attributes"]["published_at"],
            "response_time_min": job["attributes"]["response_time_in_days"].get("min"),
            "response_time_max": job["attributes"]["response_time_in_days"].get("max"),
            "applications_count": job["attributes"]["applications_count"],
            "public_url": job["links"]["public_url"]
        }
        structured_data.append(job_data)
    
    return pd.DataFrame(structured_data)

df_jobs = transform_jobs_to_dataframe(jobs)

def get_seniority_type(seniority):
    return seniority['data']['type']

df_jobs['seniority_type'] = df_jobs['seniority'].apply(get_seniority_type)

seniority_index = df_jobs.columns.get_loc('seniority')

df_jobs.insert(seniority_index + 1, 'seniority_type', df_jobs.pop('seniority_type'))

df_jobs


In [ ]:
## Los trabajos de backend tienen salarios mas altos.